# Transit Performance Analysis

This notebook analyzes courier shipment transit data to evaluate
delivery efficiency, facility usage, and overall network performance.

The goal is to flatten shipment data, calculate transit metrics,
handle real-world data issues, and generate detailed and summary reports.


## Problem Understanding

The dataset contains shipment tracking information in JSON format.
Each shipment includes multiple tracking events such as pickup,
facility arrival, in-transit, out-for-delivery, and delivery.

The task is to:
- Understand shipment movement
- Measure transit efficiency
- Analyze facility touchpoints
- Evaluate delivery performance


## Approach

1. Load and explore JSON data
2. Flatten nested shipment and event data
3. Parse timestamps in multiple formats
4. Identify pickup and delivery events
5. Calculate transit and facility metrics
6. Handle missing and inconsistent data
7. Generate detailed and summary CSV outputs


In [1]:
import json
import pandas as pd
import numpy as np
from datetime import datetime


In [3]:
with open("data/dataset.json.json", "r") as f:
    data = json.load(f)

print("Total shipments:", len(data))


Total shipments: 99


In [4]:
def parse_timestamp(ts):
    if ts is None:
        return None
    if isinstance(ts, dict) and "$numberLong" in ts:
        return datetime.fromtimestamp(int(ts["$numberLong"]) / 1000)
    try:
        return datetime.fromisoformat(ts.replace("Z", ""))
    except:
        return None


In [5]:
rows = []

for item in data:
    track_details = item.get("trackDetails", [])
    if not track_details:
        continue

    shipment = track_details[0]

    # Shipment-level info
    tracking_number = shipment.get("trackingNumber")
    carrier_code = shipment.get("carrierCode")

    service = shipment.get("service", {})
    service_type = service.get("type")
    service_desc = service.get("description", "")

    package = shipment.get("packageWeight", {})
    package_weight = package.get("value")
    package_unit = package.get("unitOfMeasure")

    packaging_type = shipment.get("packaging", {}).get("type")

    origin = shipment.get("shipperAddress", {})
    destination = shipment.get("destinationAddress", {})

    events = shipment.get("events", [])

    # Initialize metrics
    pickup_time = None
    delivery_time = None
    facilities = set()
    facility_times = []
    num_in_transit_events = 0
    out_for_delivery = 0

    in_transit_types = {"IT", "AR", "DP"}

    for e in events:
        event_type = str(e.get("eventType", "")).upper()
        desc = str(e.get("eventDescription", "")).lower()
        arrival = str(e.get("arrivalLocation", "")).upper()
        ts = parse_timestamp(e.get("timestamp"))

        # Pickup
        if pickup_time is None and (event_type == "PU" or "pickup" in desc):
            pickup_time = ts

        # Delivery
        if event_type == "DL" or "delivered" in desc:
            delivery_time = ts

        # Facility detection
        if "FACILITY" in arrival and arrival:
            facilities.add(arrival)
            if ts:
                facility_times.append(ts)

        # In-transit events
        if event_type in in_transit_types:
            num_in_transit_events += 1

        # Out for delivery
        if event_type == "OD" or "out for delivery" in desc:
            out_for_delivery += 1

    # Transit time
    if pickup_time and delivery_time:
        total_transit_hours = (delivery_time - pickup_time).total_seconds() / 3600
    else:
        total_transit_hours = None

    # Inter-facility transit time
    inter_facility_hours = 0
    facility_times = sorted(facility_times)
    for i in range(1, len(facility_times)):
        inter_facility_hours += (
            facility_times[i] - facility_times[i - 1]
        ).total_seconds() / 3600

    # Average hours per facility
    avg_hours_per_facility = (
        total_transit_hours / len(facilities)
        if total_transit_hours and len(facilities) > 0
        else None
    )

    # Express service flag
    is_express = (
        True
        if service_type and "EXPRESS" in service_type.upper()
        else False
    )

    delivery_location_type = shipment.get("deliveryLocationType")

    rows.append({
        "tracking_number": tracking_number,
        "service_type": service_type,
        "carrier_code": carrier_code,
        "package_weight_kg": package_weight,
        "packaging_type": packaging_type,
        "origin_city": origin.get("city"),
        "origin_state": origin.get("stateOrProvinceCode"),
        "origin_pincode": origin.get("postalCode"),
        "destination_city": destination.get("city"),
        "destination_state": destination.get("stateOrProvinceCode"),
        "destination_pincode": destination.get("postalCode"),
        "pickup_datetime_ist": pickup_time,
        "delivery_datetime_ist": delivery_time,
        "total_transit_hours": total_transit_hours,
        "num_facilities_visited": len(facilities),
        "num_in_transit_events": num_in_transit_events,
        "time_in_inter_facility_transit_hours": inter_facility_hours,
        "avg_hours_per_facility": avg_hours_per_facility,
        "is_express_service": is_express,
        "delivery_location_type": delivery_location_type,
        "num_out_for_delivery_attempts": out_for_delivery,
        "first_attempt_delivery": out_for_delivery == 1,
        "total_events_count": len(events)
    })


In [9]:
shipment = data[0]["trackDetails"][0]

print("Tracking Number:", shipment.get("trackingNumber"))
print("Service Type   :", shipment.get("service", {}).get("type"))
print("Carrier Code   :", shipment.get("carrierCode"))
print("Package Weight :", shipment.get("packageWeight"))
print("Packaging Type :", shipment.get("packaging", {}).get("type"))

print("\n--- EVENTS ---")
for e in shipment.get("events", []):
    print("Event Type :", e.get("eventType"))
    print("Timestamp :", parse_timestamp(e.get("timestamp")))
    print("Description:", e.get("eventDescription"))
    print("Arrival Location:", e.get("arrivalLocation"))
    print("City:", e.get("address", {}).get("city"))
    print("--------------------")


Tracking Number: 391128701026
Service Type   : FEDEX_EXPRESS_SAVER
Carrier Code   : FDXE
Package Weight : {'units': 'KG', 'value': 14}
Packaging Type : YOUR_PACKAGING

--- EVENTS ---
Event Type : DL
Timestamp : 2020-03-20 13:37:00
Description: Delivered
Arrival Location: DELIVERY_LOCATION
City: Gurgaon
--------------------
Event Type : OD
Timestamp : 2020-03-20 10:16:00
Description: On FedEx vehicle for delivery
Arrival Location: VEHICLE
City: MANESAR
--------------------
Event Type : IT
Timestamp : 2020-03-20 09:18:00
Description: In transit
Arrival Location: FEDEX_FACILITY
City: GURGAON
--------------------
Event Type : AR
Timestamp : 2020-03-20 08:46:00
Description: At local FedEx facility
Arrival Location: DESTINATION_FEDEX_FACILITY
City: MANESAR
--------------------
Event Type : IT
Timestamp : 2020-03-20 01:04:00
Description: In transit
Arrival Location: FEDEX_FACILITY
City: GURGAON
--------------------
Event Type : IT
Timestamp : 2020-03-19 23:15:00
Description: In transit
Arriva

In [10]:
df[[
    "tracking_number",
    "pickup_datetime_ist",
    "delivery_datetime_ist",
    "total_transit_hours",
    "num_facilities_visited",
    "num_out_for_delivery_attempts"
]]



,tracking_number,pickup_datetime_ist,delivery_datetime_ist,total_transit_hours,num_facilities_visited,num_out_for_delivery_attempts
0,391128701026,2020-03-16 15:44:00,2020-03-20 13:37:00,93.883333,3,1
1,390901883808,2020-03-06 16:07:00,2020-03-09 19:50:00,75.716667,3,2
2,391128749178,2020-03-16 15:44:00,2020-03-19 15:29:00,71.750000,3,1
3,390807986805,2020-03-03 16:19:00,2020-03-07 14:24:00,94.083333,2,1
4,390948921190,2020-03-09 15:12:00,2020-03-13 14:44:00,95.533333,3,1
...,...,...,...,...,...,...
94,280439181099,2021-06-16 19:22:00,2021-06-24 18:22:00,191.000000,3,1
95,281222569500,2021-07-09 14:35:00,2021-07-15 18:23:00,147.800000,3,0
96,280307632740,2021-06-14 19:31:00,2021-06-16 16:30:00,44.983333,3,1
97,280307633276,2021-06-14 19:31:00,2021-06-16 16:30:00,44.983333,3,1


In [6]:
df = pd.DataFrame(rows)
df.to_csv("transit_performance_detailed.csv", index=False)

df.head()


,tracking_number,service_type,carrier_code,package_weight_kg,packaging_type,origin_city,origin_state,origin_pincode,destination_city,destination_state,...,total_transit_hours,num_facilities_visited,num_in_transit_events,time_in_inter_facility_transit_hours,avg_hours_per_facility,is_express_service,delivery_location_type,num_out_for_delivery_attempts,first_attempt_delivery,total_events_count
0,391128701026,FEDEX_EXPRESS_SAVER,FDXE,14.0,YOUR_PACKAGING,Bangalore,KA,None,Gurgaon,HR,...,93.883333,3,7,82.400000,31.294444,True,RESIDENCE,1,True,11
1,390901883808,FEDEX_EXPRESS_SAVER,FDXE,14.0,YOUR_PACKAGING,Bangalore,KA,None,Bangalore,KA,...,75.716667,3,7,60.133333,25.238889,True,RECEPTIONIST_OR_FRONT_DESK,2,False,12
2,391128749178,FEDEX_EXPRESS_SAVER,FDXE,14.0,YOUR_PACKAGING,Bangalore,KA,None,Ahmedabad,GJ,...,71.750000,3,8,56.433333,23.916667,True,RESIDENCE,1,True,12
3,390807986805,FEDEX_EXPRESS_SAVER,FDXE,14.0,YOUR_PACKAGING,Bangalore,KA,None,New Delhi,DL,...,94.083333,2,6,81.616667,47.041667,True,RESIDENCE,1,True,10
4,390948921190,FEDEX_EXPRESS_SAVER,FDXE,14.0,YOUR_PACKAGING,Bangalore,KA,None,Delhi,DL,...,95.533333,3,7,83.000000,31.844444,True,RESIDENCE,1,True,11


In [7]:
summary = {
    "total_shipments_analyzed": len(df),
    "avg_transit_hours": df["total_transit_hours"].mean(),
    "median_transit_hours": df["total_transit_hours"].median(),
    "std_dev_transit_hours": df["total_transit_hours"].std(),
    "min_transit_hours": df["total_transit_hours"].min(),
    "max_transit_hours": df["total_transit_hours"].max(),
    "avg_facilities_per_shipment": df["num_facilities_visited"].mean(),
    "median_facilities_per_shipment": df["num_facilities_visited"].median(),
    "mode_facilities_per_shipment": df["num_facilities_visited"].mode()[0]
}

summary_df = pd.DataFrame([summary])
summary_df.to_csv("transit_performance_summary.csv", index=False)

summary_df


,total_shipments_analyzed,avg_transit_hours,median_transit_hours,std_dev_transit_hours,min_transit_hours,max_transit_hours,avg_facilities_per_shipment,median_facilities_per_shipment,mode_facilities_per_shipment
0,99,94.006734,93.25,64.829749,15.333333,544.283333,2.828283,3.0,3
